In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tifffile as tff
import scipy
from scipy import ndimage

import dask.array as da
import dask_image
from dask_image import ndfilters
from dask_image import ndmorph

from nima import nima

import skimage
import skimage.io
import skimage.filters
import zarr

from nima import utils

%load_ext autoreload
%autoreload 2

store = tff.imread("/home/dati/dt-clop3/data/210920/flatxy.tf8", aszarr=True)

zc1a = zarr.open(store)
zc1a.info

In [ ]:
dd = da.from_zarr(store)
dd

In [ ]:
img = dd[0, 0]
plt.imshow(img, vmax=60)

In [ ]:
bg, bgs, bgfigs = nima.bg(img.compute())

In [ ]:
bg, utils.bg(img.compute())

In [ ]:
pp = da.mean(
    dask_image.ndfilters.maximum_filter(dd[0:4000:20, 0], size=(100, 1, 1)), axis=0
)

In [ ]:
ppp = pp.compute()

plt.imshow(skimage.filters.gaussian(ppp, 100))

In [ ]:
m = img < skimage.filters.threshold_mean(img)
skimage.filters.threshold_mean((img * m).clip(np.min(img))).compute()

In [ ]:
plt.imshow(img < skimage.filters.threshold_triangle(img))

In [ ]:
from dask_image import ndmorph


def dabg(di):
    m = di < skimage.filters.threshold_mean(di)
    m1 = di < skimage.filters.threshold_mean((di * m).clip(np.min(di)))
    m2 = ndmorph.binary_dilation(~m1)
    return da.ma.masked_array(di, mask=~m1)

In [ ]:
def bg(im):
    m = im < skimage.filters.threshold_mean(im)
    m1 = im < skimage.filters.threshold_mean((im * m).clip(np.min(im)))
    m2 = skimage.morphology.binary_dilation(~m1, footprint=np.ones([1, 1]))
    # m2 = im < skimage.filters.threshold_triangle(np.ma.masked_array(im, mask=~m))
    return np.ma.masked_array(im, mask=m2)

In [ ]:
dabg(img).compute()

In [ ]:
flat = np.ma.mean(dabg(dd[333:500:1, 0]).compute(), axis=0)

In [ ]:
plt.imshow(flat)

### threshold mean clipping to min()

In [ ]:
tff.imshow(bg(dd[0, 0]))

In [ ]:
plt.hist(bg(img).ravel())

In [ ]:
[np.ma.median(bg(dd[i, 0])) for i in range(10)]

In [ ]:
%%time
utils.bg(img.compute(), bgmax=40)

In [ ]:
%%time
np.ma.mean(bg(img.compute()))

In [ ]:
%%time
utils.bg2(img.compute(), bgmax=60)

In [ ]:
utils._bgmax(img.compute(), step=0.1)

In [ ]:
utils.pbar.unregister()

### masked array (ma)

In [ ]:
a = np.ma.masked_array([1, 4, 3], mask=[False, False, True])
b = np.ma.masked_array([10, 2, 6], mask=[False, True, False])

np.ma.median([a, b], axis=0)

In [ ]:
np.ma.median(np.ma.stack([a, b]), axis=0)

In [ ]:
img

In [ ]:
f3 = img > skimage.filters.threshold_local(img.compute(), 601)

In [ ]:
f3

In [ ]:
img[~f3].mean().compute()

In [ ]:
m1 = np.ma.masked_greater(img, 15)

## generat

In [ ]:
image = "bias + noise + dark + flat * (sky + obj)"

In [ ]:
image

- **bias**: generate a wave-like shape along x.
- **noise**: random number will do.
- **dark**: simply a scalar value.
- **flat**: generate some 2D parabolic shape.
- **obj**: circles-ellipsis. (MAYBE: like finite fractals to compare segmentation).
- **sky**: None | some blurred circle-ellipsoid coincident and not with some obj.

fg_prj := 

bg_prj := 

In [ ]:
from nima import generat

In [ ]:
flat = generat.gen_flat(nrows=29)
img = plt.imshow(flat)
plt.colorbar(img, location="top", shrink=0.5)

In [ ]:
# %%timeit
bias = generat.gen_bias()
flat = generat.gen_flat()
objs = generat.gen_objs()
frame = generat.gen_frame(objs)
# plt.imshow(frame)

In [ ]:
objs = generat.gen_objs()
frame = generat.gen_frame(objs, bias, flat)
# plt.imshow(frame)

In [ ]:
bias = np.zeros((128, 128))
flat = np.ones((128, 128))

stack = np.stack(
    [
        generat.gen_frame(
            generat.gen_objs(max_fluor=10), bias, flat, noise_sd=10, sky=10
        )
        for i in range(1000)
    ]
)

In [ ]:
stat_bg = []
for s in stack[:]:
    stat_bg.append(utils.bg(s)[0])

In [ ]:
plt.hist(stat_bg)
np.mean(stat_bg), np.std(stat_bg)

bg2 was less robust with small signal

## what is the best projection for flat calculation? 

In [ ]:
bias = generat.gen_bias()
flat = generat.gen_flat()
stack = np.stack(
    [
        generat.gen_frame(generat.gen_objs(max_fluor=20), bias, flat, noise_sd=1, sky=2)
        for i in range(1000)
    ]
)

In [ ]:
def splot(stack, num=4):
    f, axs = plt.subplots(1, num)
    for i in range(num):
        axs[i].imshow(stack[np.random.randint(len(stack))])


splot(stack)

In [ ]:
def diff_plot(im, flat, title):
    f, axs = plt.subplots(1, 2)
    diff = im / im.mean() - flat
    skimage.io.imshow(diff, ax=axs[0])
    axs[1].hist(diff.ravel())
    f.suptitle(title)
    return diff.mean(), diff.std()


def prj_plot(t_prj, title, sigma=128 / 11):
    im = ndimage.gaussian_filter(t_prj, sigma=sigma)
    return diff_plot(im, flat, title)


def prj(stack, func):
    t_prj = func(stack, axis=0)
    return prj_plot(t_prj, func.__name__)


prj(stack, np.max)

In [ ]:
prj(stack, np.mean)

In [ ]:
prj(stack, np.median)

In [ ]:
from functools import partial

p999 = partial(np.percentile, q=99.9)
p999.__name__ = "percentile 99.9%"

prj(stack, p999)

In [ ]:
im = np.mean(
    ndfilters.median_filter(
        da.from_array(stack[:100] - bias), size=(32, 16, 16)
    ).compute(),
    axis=0,
)
prj_plot(im, "dd", sigma=7)

#### Knowing the Bias.

In [ ]:
prj(stack - bias, p999)

In [ ]:
prj(stack - bias, np.mean)

### Using fg and bg masks?

And assuming we know the bias of the camera.

In [ ]:
def mask_plane(plane, bg_ave=2, bg_std=1.19, erf_pvalue=0.01):
    p = utils.prob(plane, bg_ave, bg_std)
    p = ndimage.median_filter(p, size=2)
    mask = p > erf_pvalue
    mask = skimage.morphology.remove_small_holes(mask)
    return np.ma.masked_array(plane, mask=~mask), np.ma.masked_array(plane, mask=mask)


plt.imshow(mask_plane(stack[113], *utils.bg(stack[113]))[0])

In [ ]:
bgs, fgs = list(zip(*[mask_plane(s - bias, *utils.bg(s - bias)) for s in stack]))

splot(bgs)

In [ ]:
t_prj = np.ma.mean(np.ma.stack(bgs), axis=0)
prj_plot(t_prj, "Bg mean", sigma=3)

In [ ]:
t_prj = np.ma.max(np.ma.stack(fgs), axis=0)
prj_plot(t_prj, "Fg max (bias known)", sigma=2)

In [ ]:
bgs, fgs = list(zip(*[mask_plane(s, *utils.bg(s)) for s in stack]))

bg_prj1 = np.ma.mean(np.ma.stack(bgs[:]), axis=0)
fg_prj1 = np.ma.mean(np.ma.stack(fgs[:]), axis=0)
im = fg_prj1 - bg_prj1
diff_plot(ndimage.gaussian_filter(im, 1), flat, "Bg mean - fg mean")

In [ ]:
bg_prj = np.ma.mean(bgs, axis=0)
fg_prj = np.ma.max(fgs, axis=0)
# im = ndimage.median_filter(bg_prj-fg_prj, size=60) #- 2 * flat
im = ndimage.gaussian_filter(bg_prj - fg_prj, sigma=14)  # - 2 * flat

diff_plot(im, flat, "m")

In [ ]:
t_prj = np.ma.max(fgs, axis=0)
prj_plot(t_prj, "Fg MAX", sigma=13)

In [ ]:
eflat = bg_prj - fg_prj
eflat /= eflat.mean()
eflat = ndimage.gaussian_filter(eflat, sigma=13)

diff_plot(eflat, flat, "eflat")

## When bias and flat are unknown...

- bias = bg_prj - sky * flat
- bias = fg_prj - flat

sky * flat - flat = bg_prj - fg_prj

In [ ]:
diff_plot((bg_prj1 - bias) / 2, flat, "")

In [ ]:
plt.imshow((im - bias) / (im - bias).mean() - flat)
plt.colorbar()

## cfr. nima.bg

In [ ]:
# r = nima.bg((stack[113] - bias) / flat)
r = nima.bg(stack[111])

In [ ]:
r[1].mean(), r[1].std()

In [ ]:
utils.bg(stack[111])

In [ ]:
bias.mean() + 2

## geometric mean

In [ ]:
vals = [0.8, 0.1, 0.3, 0.1, 0.8, 0.8, 0.8, 0.1, 0.8]

np.median(vals), scipy.stats.gmean(vals), np.mean(vals)

In [ ]:
(0.8 * 0.8 * 0.8 * 0.8 * 0.1) ** (1 / 5)